In [1]:
#Importing all required libraries
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
import shutil
from shutil import rmtree
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#Transfer learning using pre-trained Inception Model
path_inception = f"{getcwd()}/drive/My Drive/Colab Notebooks/Dog classification/inception_v3_weights.h5"
checkpoint_path = f"{getcwd()}/drive/My Drive/Colab Notebooks/Dog classification/checkpoint.h5"

# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = path_inception

print('Creating Inception model')
pre_trained_model = InceptionV3(input_shape = (150,150,3),
                               include_top = False,
                               weights = None)

print('Disabling model learning')
# Make all the layers in the pre-trained model non-trainable
for layer in pre_trained_model.layers:
    layers.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

#Reading training data csv and concatenating filenames with extension ".jpg"
training_data = pd.read_csv(f"{getcwd()}/drive/My Drive/Colab Notebooks/Dog classification/labels.csv")
training_data['id'] = training_data['id'] + ".jpg"

Creating Inception model
Disabling model learning


In [3]:
# Define a Callback class that stops training once accuracy reaches 97.0%
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.97):
            print("\nReached 97.0% accuracy so cancelling training!")
            self.model.stop_training = True

In [4]:
print('Adding more Dense layers')
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)

x = layers.Dense(512, activation = 'relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)  

x = layers.Dense(256, activation = 'relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)    

# Add a final softmax layer for classification
x = layers.Dense(120, activation = 'softmax')(x)           

model = Model(pre_trained_model.input, x) 

print('Loading weights into the model')
#pre_trained_model.load_weights(checkpoint_path)
pre_trained_model.load_weights(path_inception)

print('Model compiling')
model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])
print('Model compiled')

Adding more Dense layers
Loading weights into the model
Model compiling
Model compiled


In [5]:
# model = load_model(checkpoint_path)

In [6]:
# Add our data-augmentation parameters to ImageDataGenerator
train_dir = f"{getcwd()}/drive/My Drive/Colab Notebooks/Dog classification/train/"
BatchSize = 500
Epochs = 1

print('Performing Data Augmentation')
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

train_generator = train_datagen.flow_from_dataframe(dataframe = training_data, 
                                                    directory = f"{getcwd()}/drive/My Drive/Colab Notebooks/Dog classification/train_images", 
                                                    x_col = "id", y_col = "breed", 
                                                    class_mode = "categorical", 
                                                    target_size = (150, 150), 
                                                    batch_size = BatchSize)

callbacks = myCallback()

print('Training the model')
history = model.fit(
            train_generator,
            steps_per_epoch = np.ceil(len(training_data)/BatchSize),
            epochs = Epochs, 
            verbose = 1,
            callbacks = [callbacks])

Performing Data Augmentation
Found 10222 validated image filenames belonging to 120 classes.
Training the model


KeyboardInterrupt: ignored

In [ ]:
model.save(checkpoint_path)

In [ ]:
#Output of testing folder
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)